In [2]:
# Check device number
!ls -ltrh /dev/video*

crw-rw----+ 1 root video 81, 1 Aug 28 09:18 /dev/video1
crw-rw----+ 1 root video 81, 0 Aug 28 09:18 /dev/video0


In [2]:
from ultralytics import YOLO
import torch
import os

print(torch.__version__, "cuda:", torch.cuda.is_available())

#make ft16 tensorrt file
model = YOLO("yolo.pt", task="pose")    # 또는 task="detect"

model.export(format="engine", half = True)

os.rename("yolo.engine", "yolo_fp16.engine")
os.rename("yolo.onnx", "yolo_fp16.engine")


2.8.0 cuda: True
WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.187 🚀 Python-3.10.12 torch-2.8.0 CUDA:0 (Orin, 7620MiB)
YOLO11n-pose summary (fused): 109 layers, 2,956,000 parameters, 0 gradients, 7.8 GFLOPs

PyTorch: starting from 'yolo.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 68, 8400) (6.0 MB)
requirements: Ultralytics requirement ['onnxruntime-gpu'] not found, attempting AutoUpdate...


ERROR: Could not find a version that satisfies the requirement onnxruntime-gpu (from versions: none)


WARNING ⚠️ Retry 1/2 failed: Command 'pip install --no-cache-dir "onnxruntime-gpu" ' returned non-zero exit status 1.


ERROR: No matching distribution found for onnxruntime-gpu
ERROR: Could not find a version that satisfies the requirement onnxruntime-gpu (from versions: none)


WARNING ⚠️ Retry 2/2 failed: Command 'pip install --no-cache-dir "onnxruntime-gpu" ' returned non-zero exit status 1.
WARNING ⚠️ requirements: ❌ Command 'pip install --no-cache-dir "onnxruntime-gpu" ' returned non-zero exit status 1.


ERROR: No matching distribution found for onnxruntime-gpu



ONNX: starting export with onnx 1.12.0 opset 19...
ONNX: slimming with onnxslim 0.1.59...
WARNING ⚠️ ONNX: simplifier failure: module 'onnx.helper' has no attribute 'get_all_tensor_dtypes'
ONNX: export success ✅ 8.5s, saved as 'yolo.onnx' (11.6 MB)

TensorRT: starting export with TensorRT 10.3.0...
[08/28/2025-07:41:07] [TRT] [I] [MemUsageChange] Init CUDA: CPU +1, GPU +0, now: CPU 749, GPU 4495 (MiB)
[08/28/2025-07:41:10] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +928, GPU +696, now: CPU 1720, GPU 5226 (MiB)
[08/28/2025-07:41:10] [TRT] [I] ----------------------------------------------------------------
[08/28/2025-07:41:10] [TRT] [I] Input filename:   yolo.onnx
[08/28/2025-07:41:10] [TRT] [I] ONNX IR version:  0.0.9
[08/28/2025-07:41:10] [TRT] [I] Opset version:    19
[08/28/2025-07:41:10] [TRT] [I] Producer name:    pytorch
[08/28/2025-07:41:10] [TRT] [I] Producer version: 2.8.0
[08/28/2025-07:41:10] [TRT] [I] Domain:           
[08/28/2025-07:41:10] [TRT] [I] Mod

In [3]:
from flask import Flask, Response
from ultralytics import YOLO
import cv2

# YOLO 모델 불러오기 (best.pt or .engine 가능)
model = YOLO("yolo_fp16.engine", task="pose")

c_index = 0 #Camera index can be changed

cap = cv2.VideoCapture(c_index) 
assert cap.isOpened(), "Camera open failed."

app = Flask(__name__)

def gen_frames():
    while True:
        ret, frame = cap.read() 
        if not ret:
            break

        # YOLO inference
        results = model.predict(source=frame, imgsz=640,verbose=False)
        annotated = results[0].plot()  # draw inference results

        # JPEG encoding
        ret, buffer = cv2.imencode('.jpg', annotated)
        if not ret:
            continue #If encoding is failed, skip

        # MJPEG streaming: MJPEG = Motion JPEG
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + buffer.tobytes() + b'\r\n')

@app.route('/')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__":
    try:
        app.run(host="0.0.0.0", port=8000, debug=False, use_reloader=False)
    finally:
        cap.release()
        cv2.destroyAllWindows()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.30.1.50:8000
Press CTRL+C to quit


Loading yolo_fp16.engine for TensorRT inference...
[08/28/2025-09:18:24] [TRT] [I] Loaded engine size: 9 MiB
[08/28/2025-09:18:24] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +11, now: CPU 0, GPU 16 (MiB)


172.30.1.93 - - [28/Aug/2025 09:18:29] "GET / HTTP/1.1" 200 -
